### Changing Directory

In [1]:
import os

In [2]:
%pwd

'm:\\Chicken-Disease\\research'

In [3]:
os.chdir("../")     # change directory, go back once

In [4]:
%pwd

'm:\\Chicken-Disease'

In [5]:
# entity is the returun type of a function
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Creating Data Ingestion Notebook

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # method to download file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    # extracting, after downloading
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


### Constructing the pipeline
Pipeline is a way to break down a series of complicated data processing events into smaller, more manageable steps. Its meaning depends on its usage.
Ex: In pandas, pipeline feature allows us to string together various user-defined functions. In scikit-learn, pipeline is a utility for orchestrating machine learning operations.

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-22 22:05:32,165: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-22 22:05:32,189: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 22:05:32,191: INFO: common: created directory at: artifacts]
[2023-07-22 22:05:32,194: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-22 22:06:51,541: INFO: 3722065165: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24163944
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "602d0e7926d491fffbdc322339392c659ce28448357eeb87642f475694d846ec"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4D80:5A0C:FC70B:12D54C:64BC054E
Accept-Ranges: bytes
Date: Sat, 22 Jul 2023 16:35:36 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4730-BOM
X-Cache: MISS